# Automated setup for WKbeam and WKBacca simulation

This module allows for the computation of the quasilinear radio-frequency diffusion tensor components based on the inputs
of a LUKE calculation and wave propagation as calculated from WKBeam.


In [10]:
# Import the necessary modules
import os
import sys
import subprocess

import numpy as np
import matplotlib.pyplot as plt
from Simul_prep_aux import create_workspace, update_config

## Create new folder for the WKBeam calculation 

In [11]:
calc_path           = '/home/devlamin/WKbeam_simulations'
new_calc_folder     = 'TCV_85352_1.3_fluct'
folders             = ['input', 'output', 'plots', 'configs']


In [12]:
create_workspace(calc_path, new_calc_folder, folders = folders)

New folder TCV_85352_1.3_fluct created for the WKBeam calculation at /home/devlamin/WKbeam_simulations/TCV_85352_1.3_fluct
Subfolder input created at /home/devlamin/WKbeam_simulations/TCV_85352_1.3_fluct/input
Subfolder output created at /home/devlamin/WKbeam_simulations/TCV_85352_1.3_fluct/output
Subfolder plots created at /home/devlamin/WKbeam_simulations/TCV_85352_1.3_fluct/plots
Subfolder configs created at /home/devlamin/WKbeam_simulations/TCV_85352_1.3_fluct/configs
Input data should be placed in the new input folder.


At this point you should put the correct ne.dat, Te.dat, topfile, fluctuations file and Lperp file in the input directory 

## Create the configuration files from the Standardconfigs and the new values

In [17]:
# Location of some standard config files
Standardconfigs_loc = '/home/devlamin/WKBacca_QL/Simul_prep/StandardConfigs'
# Location of the new values for the config files
# Usually would be in your newly created folder under the config subfolder
Newvalues_loc       = os.path.join(calc_path, new_calc_folder, 'configs')
Calculation_loc         = os.path.join(calc_path, new_calc_folder)

In [18]:
print('Creating new configuration files...')
for file in os.listdir(Newvalues_loc):
    if file.endswith('.txt'):
        kind = file.split('_')[0]
        standard_file = os.path.join(Standardconfigs_loc, kind + '.txt')
        new_values_file = os.path.join(Newvalues_loc, file)
        output_file = os.path.join(Calculation_loc, kind + '.txt')
        update_config(standard_file, new_values_file, output_file)

print('New configuration files created.')

Creating new configuration files...
New configuration files created.


## Creating bash script for WKBeam

In [19]:
Number_of_Cores = 10
WKBeam_loc = '/home/devlamin/WKBacca_QL/'

In [20]:
# Create a bash script to run the calculations
script = os.path.join(WKBeam_loc, new_calc_folder + '.sh')
with open(script, 'w') as f:
    f.write('#!/bin/bash\n')
    f.write('command1=' + Calculation_loc + '/RayTracing.txt\n')
    f.write('mpiexec -np ' + str(Number_of_Cores) + ' python3 WKBeam.py trace $command1\n')

    # Write stuff for postprocessing
    f.write('wait')
    f.write('\n')
    f.write('python3 WKBeam.py bin ' + Calculation_loc + '/Angular.txt\n')
    f.write('python3 WKBeam.py bin ' + Calculation_loc + '/Absorption.txt\n')
    f.write('python3 WKBeam.py bin ' + Calculation_loc + '/XZ.txt\n')
    f.write('python3 WKBeam.py bin ' + Calculation_loc + '/RhoThetaN.txt\n')

    f.write('wait')
    f.write('\n')
    f.write('echo "All done!"')

    # Make the script executable
    os.chmod(script, 0o755)


    # close 
    f.close()

# Create the bash script for figures
script = os.path.join(WKBeam_loc, new_calc_folder + '_figures.sh')

with open(script, 'w') as f:
    f.write('#!/bin/bash\n')
    f.write('command1=' + Calculation_loc + '/RayTracing.txt\n')
    f.write('python3 WKBeam.py plot2d ' + Calculation_loc + '/Angular.txt &\n')
    f.write('python3 WKBeam.py plotabs ' + Calculation_loc + '/Absorption.txt &\n')
    f.write('python3 WKBeam.py plotbin ' + Calculation_loc + '/output/XZ_binned.hdf5' + ' $command1 &\n')
    f.write('python3 WKBeam.py beamFluct ' + Calculation_loc + '/output/XZ_binned.hdf5' + ' $command1 &\n')

    f.write('wait')
    f.write('\n')
    f.write('echo "All done!"')

    # Make the script executable
    os.chmod(script, 0o755)

    # close
    f.close()